In [25]:
# Loading all the needed Packages
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Import classes from Class.py
from Class import *

# Data Handling

In [26]:
## PREPARING THE DATA

# Load all CSV files
Dem_Data = pd.read_excel('../Data/LoadProfile.xlsx', sheet_name='Python_Dem_Data')
Uti_Data = pd.read_excel('../Data/LoadProfile.xlsx', sheet_name='Python_Uti_Data')
Load_Data = pd.read_excel('../Data/LoadProfile.xlsx', sheet_name='Python_Load_Data')
Gen_E_Data = pd.read_excel('../Data/Generators_Existing.xlsx', sheet_name='Python_Gen_E_Data')
Gen_N_Data = pd.read_excel('../Data/Generators_New.xlsx', sheet_name='Python_Gen_N_Data')
Gen_E_Z_Data = pd.read_excel('../Data/Generators_Existing.xlsx', sheet_name='Python_Gen_E_Z_Data')
Gen_N_Z_Data = pd.read_excel('../Data/Generators_New.xlsx', sheet_name='Python_Gen_N_Z_Data')
Gen_E_OpCap_Data = pd.read_excel('../Data/GenerationProfile.xlsx', sheet_name='Python_Gen_E_OpCap_Data')
Gen_N_OpCap_Data = pd.read_excel('../Data/GenerationProfile.xlsx', sheet_name='Python_Gen_N_OpCap_Data')
Trans_Data = pd.read_excel('../Data/Transmission.xlsx', sheet_name='Python_Trans_Data')
Trans_Line_From_Z = pd.read_excel('../Data/Transmission.xlsx', sheet_name='Python_Line_From_Z_Data')
Trans_Line_To_Z = pd.read_excel('../Data/Transmission.xlsx', sheet_name='Python_Line_To_Z_Data')
Trans_Z_Connected_To_Z = pd.read_excel('../Data/Transmission.xlsx', sheet_name='Python_Z_Connected_To_Z_Data')


# Export the needed matrices
Dem = np.array(Dem_Data)    # Demand profile
Uti = np.transpose(np.array(Uti_Data))   # Utility profile
Load_Z = np.array(Load_Data)   # Load Zone
Gen_E_OpCost = np.array(Gen_E_Data['Cost'])   # Existing Generators Operational Cost
Gen_N_OpCost = np.array(Gen_N_Data['Cost'])  # New Generators Operational Cost
Gen_N_MaxInvCap = np.array(Gen_N_Data['MaxInv (MW)'])  # Maximum New Generators Capacity Investment (MW)
Gen_N_InvCost = np.array(Gen_N_Data['C_CapInv ($/MW)'])  # New Generators Investment Cost ($/MW)
Gen_E_Tech = np.array(Gen_E_Data['Technology'])  # Existing Generators Technology
Gen_N_Tech = np.array(Gen_N_Data['Technology'])  # New Generators Technology
Gen_E_Z = np.array(Gen_E_Z_Data)  # Existing Generators Zone
Gen_N_Z = np.array(Gen_N_Z_Data)  # New Generators Zone
Gen_E_OpCap = np.array(Gen_E_OpCap_Data)  # Maximum Capacity of Existing Generators (Hourly profile if RES, Max capacity otherwise)
Gen_N_OpCap = np.array(Gen_N_OpCap_Data)  # Maximum Capacity of New Generators (Hourly profile if RES, Max capacity otherwise)
Trans_React = np.array(Trans_Data['Reactance'])  # Transmission Reactance
Trans_Cap = np.array(Trans_Data['Capacity [MW]'])  # Transmission Capacity
Trans_Line_From_Z = np.array(Trans_Line_From_Z)  # Mapping the origine zone for each transmission line
Trans_Line_To_Z = np.array(Trans_Line_To_Z)  # Mapping the destination zone for each transmission line
Trans_Z_Connected_To_Z = np.array(Trans_Z_Connected_To_Z)  # Mapping the connected zones for each zone

# Fix the shape of matrices with only one column   
Gen_E_OpCost = Gen_E_OpCost.reshape((Gen_E_OpCost.shape[0], 1))
Gen_N_OpCost = Gen_N_OpCost.reshape((Gen_N_OpCost.shape[0], 1))
Gen_N_MaxInvCap = Gen_N_MaxInvCap.reshape((Gen_N_MaxInvCap.shape[0], 1))
Gen_N_InvCost = Gen_N_InvCost.reshape((Gen_N_InvCost.shape[0], 1))
Gen_E_Tech = Gen_E_Tech.reshape((Gen_E_Tech.shape[0], 1))
Gen_N_Tech = Gen_N_Tech.reshape((Gen_N_Tech.shape[0], 1))
Trans_React = Trans_React.reshape((Trans_React.shape[0], 1))
Trans_Cap = Trans_Cap.reshape((Trans_Cap.shape[0], 1))


In [27]:
## DATA INDEX

# Create a Dataframe to store the name of each vector/matrix we will use, their size and their content
Data_df = pd.DataFrame(columns=['Name', 'Size', 'Content'])
Data_df['Name'] = ['Dem', 'Uti', 'Load_Z', 'Gen_E_OpCost', 'Gen_N_OpCost', 'Gen_N_MaxInvCap', 'Gen_N_InvCost', 'Gen_E_Tech', 'Gen_N_Tech', 'Gen_E_Z', 'Gen_N_Z', 'Gen_E_OpCap', 'Gen_N_OpCap', 'Trans_React', 'Trans_Cap', 'Trans_Line_From_Z', 'Trans_Line_To_Z', 'Trans_Z_Connected_To_Z']
Data_df['Size'] = [Dem.shape, Uti.shape, Load_Z.shape, Gen_E_OpCost.shape, Gen_N_OpCost.shape, Gen_N_MaxInvCap.shape, Gen_N_InvCost.shape, Gen_E_Tech.shape, Gen_N_Tech.shape, Gen_E_Z.shape, Gen_N_Z.shape, Gen_E_OpCap.shape, Gen_N_OpCap.shape, Trans_React.shape, Trans_Cap.shape, Trans_Line_From_Z.shape, Trans_Line_To_Z.shape, Trans_Z_Connected_To_Z.shape ]
Data_df['Content'] = ['Demand for each load for each hour of the investment problem',
                     'Utility for each load for one hour',
                     'Zone of each load',
                     'Operationnal cost of each existing generator',
                     'Operationnal cost of each new generator',
                     'Maximum capacity investment of each new generator',
                     'Unit Investment cost of each new generator',
                     'Technology of each existing generator',
                     'Technology of each new generator',
                     'Zone of each existing generator',
                     'Zone of each new generator',
                     'Maximum operationnal capacity of each existing energy source for each hour of the investment problem (Hourly profile if RES, Max capacity otherwise)',
                     'Maximum operationnal capacity of each new energy source for each hour of the investment problem (Hourly profile if RES, Max capacity otherwise)',
                     'Transmission Reactance',
                     'Transmission Capacity',
                     'Origine zone of each transmission line',
                     'Destination zone of each transmission line',
                     'Connected zones for each zone']
Data_df

,Name,Size,Content
0,Dem,"(3600, 17)",Demand for each load for each hour of the inve...
1,Uti,"(17, 1)",Utility for each load for one hour
2,Load_Z,"(2, 17)",Zone of each load
3,Gen_E_OpCost,"(16, 1)",Operationnal cost of each existing generator
4,Gen_N_OpCost,"(16, 1)",Operationnal cost of each new generator
5,Gen_N_MaxInvCap,"(16, 1)",Maximum capacity investment of each new generator
6,Gen_N_InvCost,"(16, 1)",Unit Investment cost of each new generator
7,Gen_E_Tech,"(16, 1)",Technology of each existing generator
8,Gen_N_Tech,"(16, 1)",Technology of each new generator
9,Gen_E_Z,"(2, 16)",Zone of each existing generator


In [28]:
## PARAMETERS DEFINITION

# Time
H = 24          # Hours in a day
D = 5           # Typical days in a year
Y = 30          # Years of the investment timeline
N = H*D*Y       # Number of hours in the investment timeline    

# Number of loads and generators
N_dem = len(Dem[0,:])       # Number of loads
N_gen_E = len(Gen_E_OpCost)   # Number of existing generators
N_gen_N = len(Gen_N_OpCost)   # Number of new generators
N_zone = len(Trans_Z_Connected_To_Z)     # Number of zones
N_line = len(Trans_Line_From_Z)   # Number of transmission lines

# Hyperparameters
B = 1000000000    # Budget for the investment problem


In [29]:
## CREATE THE PARAMETERS AND DATA OBJECTS
ParametersObj = Parameters(H, D, Y, N, N_dem, N_gen_E, N_gen_N, N_zone, N_line, B*5)
DataObj = InputData(Dem, Uti, Load_Z, Gen_E_OpCost, Gen_N_OpCost, Gen_N_MaxInvCap, Gen_N_InvCost/500, Gen_E_Tech, Gen_N_Tech, Gen_E_Z, Gen_N_Z, Gen_E_OpCap, Gen_N_OpCap, Trans_React, Trans_Cap, Trans_Line_From_Z, Trans_Line_To_Z, Trans_Z_Connected_To_Z)

# Model 1: Sequential optimization of Dispatch problem and Investment problem

### 1) Market Clearing

In [30]:
# Ruun the Market Clearing Problem
MarketClearing1 = MarketClearingModel1(ParametersObj, DataObj)

Objective value:  3491848281.35839


In [31]:
MarketClearing1.res.df

,Hour,Load Zone 0,Existing generators Zone 0,Price Zone 0,Load Zone 1,Existing generators Zone 1,Price Zone 1,Power flow line 1,Power flow line 2
0,1,1068.403919,1048.029318,40.0,710.981270,731.355870,40.0,-20.374600,20.374600
1,2,991.074122,941.074122,40.0,659.521297,709.521297,40.0,-50.000000,50.000000
2,3,943.605644,897.671500,40.0,627.932870,673.867014,40.0,-45.934144,45.934144
3,4,938.268265,889.053478,40.0,624.381052,673.595839,40.0,-49.214787,49.214787
4,5,957.731229,907.731229,40.0,637.332897,687.332897,40.0,-50.000000,50.000000
5,6,972.778082,972.778082,40.0,632.251114,632.251114,75.0,0.000000,0.000000
6,7,1074.419762,1074.419763,75.0,584.252344,584.252344,80.0,0.000000,0.000000
7,8,1049.244987,1049.244987,75.0,554.887639,554.887639,125.0,0.000000,0.000000
8,9,1075.406746,1075.406746,80.0,513.135357,513.135357,125.0,0.000000,0.000000
9,10,1151.224342,1151.224342,80.0,451.636018,451.636018,125.0,0.000000,0.000000


### 2) Investment Problem

In [32]:
# Run the investmentmodel
InvestmentPB1 = InvestmentModel1(ParametersObj, DataObj, MarketClearing1.res.DA_price)

Objective value:  5001316590.0


In [33]:
InvestmentPB1.res.df

,Technology,Invested capacity (MW)
0,Coal,200.0
1,Coal,200.0
2,Coal,200.0
3,Gas,200.0
4,Gas,0.0
5,Coal,200.0
6,Coal,200.0
7,Wind,0.0
8,Wind,0.0
9,PV,0.0


# Model 2: Integrated Bi-level optimization of dispatch problem and investment problem using KKTs

In [34]:
## CLASS FOR THE INVESTMENT PROBLEM

class Model2_EXAMPLE():
    def __init__(self, Parameters, Data, DA_Price, Model_results = 1, Guroby_results = 0):
        self.D = Data  # Data
        self.P = Parameters  # Parameters
        self.DA_Price = DA_Price  # Day-ahead price
        self.Model_results = Model_results  # Display results
        self.Guroby_results = Guroby_results  # Display guroby results
        self.var = Expando()  # Variables
        self.con = Expando()  # Constraints
        self.res = Expando()  # Results
        self._build_model() 


    def _build_variables(self):
        self.var.P_N = self.m.addMVar((self.P.N_gen_N, 1), lb=0) # Invested capacity in every new generator
        self.var.p_N = self.m.addMVar((self.P.N, self.P.N_gen_N), lb=0) # Power output per hour for every new generator


    def _build_constraints(self):
        # Capacity investment constraint
        self.con.cap_inv = self.m.addConstr(self.var.P_N <= self.D.Gen_N_MaxInvCap, name='Maximum capacity investment')

        # Max production constraint
        ratio_invest = (self.var.P_N.T / self.D.Gen_N_MaxInvCap.T) # % of the maximum investment capacity invested in each new generator, size (1, N_gen_N)
        self.ratio_invest_hourly = self.P.Sum_over_hours_gen_N * ratio_invest # Create a matrix of size (N, N_gen_N) with the % of the maximum investment capacity invested in each new generator for each hour
        self.con.max_p_N = self.m.addConstr(self.var.p_N <= self.D.Gen_N_OpCap * self.ratio_invest_hourly , name='Maximum RES production')

        # Budget constraint
        self.con.budget = self.m.addConstr(self.var.P_N.T @ self.D.Gen_N_InvCost <= self.P.B, name='Budget constraint')


    def _build_objective(self):
        revenues = ((self.var.p_N @ self.D.Gen_N_Z.T) * self.DA_Price).sum()  # don't use quicksum here because it's a <MLinExpr (3600, N_zone)>
        op_costs = gp.quicksum(self.var.p_N @ self.D.Gen_N_OpCost)
        budget_init = self.P.B
        invest_costs = self.var.P_N.T @ self.D.Gen_N_InvCost
        objective = revenues - op_costs + budget_init - invest_costs
        self.m.setObjective(objective, GRB.MAXIMIZE)


    def _display_guropby_results(self):
        self.m.setParam('OutputFlag', self.Guroby_results)
    

    def _build_model(self):
        self.m = gp.Model('Investment problem')
        self._build_variables()  
        self._build_constraints()
        self._build_objective()
        self._display_guropby_results()
        self.m.optimize()
        if self.Model_results == 1:
            self._extract_results()

    def _extract_results(self):
        # Display the objective value
        print('Objective value: ', self.m.objVal)
        
        # Display the generators the model invested in, in a dataframe
        self.res.P_N = self.var.P_N.X
        self.res.P_N = self.res.P_N.reshape((self.P.N_gen_N,1))
        self.res.df = pd.DataFrame(self.D.Gen_N_Tech, columns = ['Technology'])
        self.res.df['Invested capacity (MW)'] = self.res.P_N
        


    

        

In [35]:
class Model2():
    
    def __init__(self, Parameters, Data, DA_Price, Model_results = 1, Guroby_results = 0):
        self.D = Data  # Data
        self.P = Parameters  # Parameters
        self.DA_Price = DA_Price  # Day-ahead price
        self.Model_results = Model_results  # Display results
        self.Guroby_results = Guroby_results  # Display guroby results
        self.var = Expando()  # Variables
        self.con = Expando()  # Constraints
        self.res = Expando()  # Results
        self._build_model() 

    

    def _build_variables(self):
        self.var.d = self.m.addMVar((self.P.N, self.P.N_dem), lb=0)  # demand per hour for every load
        self.var.p_E = self.m.addMVar((self.P.N, self.P.N_gen_E), lb=0)  # power output per hour for every existing generator
        self.var.theta = self.m.addMVar((self.P.N,self.P.N_zone), lb=0)  # power flow per hour for every transmission line
        self.var.P_N = self.m.addMVar((self.P.N_gen_N, 1), lb=0) # Invested capacity in every new generator
        self.var.p_N = self.m.addMVar((self.P.N, self.P.N_gen_N), lb=0) # Power output per hour for every new generator



    def _build_constraints(self):
        # Capacity investment constraint
        self.con.cap_inv = self.m.addConstr(self.var.P_N <= self.D.Gen_N_MaxInvCap, name='Maximum capacity investment')

        # Budget constraint
        self.con.budget = self.m.addConstr(self.var.P_N.T @ self.D.Gen_N_InvCost <= self.P.B, name='Budget constraint')


        